In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

# Load file bạn đã upload
df = pd.read_csv("./customer_churn_dataset-testing-master.csv")
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0


1) Tính Accuracy, Precision, Recall, F1 cho Decision Tree và Logistic Regression
Tiền xử lý dữ liệu cho bài toán phân loại

In [4]:
# Drop customerID nếu có
if "customerID" in df.columns:
    df = df.drop(columns=["customerID"])

# Encode nhãn Churn
label = LabelEncoder()
df["Churn"] = label.fit_transform(df["Churn"])


Xử lý biến số và biến phân loại

In [5]:
# Tách numeric và categorical
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()

# One-hot cho categorical
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

df_encoded.head()


,CustomerID,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn,Gender_Male,Subscription Type_Premium,Subscription Type_Standard,Contract Length_Monthly,Contract Length_Quarterly
0,1,22,25,14,4,27,598,9,1,False,False,False,True,False
1,2,41,28,28,7,13,584,20,0,False,False,True,True,False
2,3,47,27,10,2,29,757,21,0,True,True,False,False,False
3,4,35,9,12,5,17,232,18,0,True,True,False,False,True
4,5,53,58,24,9,2,533,18,0,False,False,True,False,False


Chia train/test

In [6]:
X = df_encoded.drop("Churn", axis=1)
y = df_encoded["Churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

Huấn luyện Decision Tree

In [7]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)

Huấn luyện Logistic Regression

In [8]:
logr = LogisticRegression(max_iter=500)
logr.fit(X_train, y_train)

y_pred_lr = logr.predict(X_test)


d:\D-PROJECTS\University\khai-thac-du-lieu\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Tính các metric

In [9]:
def print_metrics(y_true, y_pred):
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1 Score :", f1_score(y_true, y_pred))

print("=== Decision Tree ===")
print_metrics(y_test, y_pred_dt)

print("\n=== Logistic Regression ===")
print_metrics(y_test, y_pred_lr)


=== Decision Tree ===
Accuracy : 0.9968155339805825
Precision: 0.996547756041427
Recall   : 0.9967116080236764
F1 Score : 0.9966296752979861

=== Logistic Regression ===
Accuracy : 0.830990291262136
Precision: 0.8112846668791839
Recall   : 0.8368957579743506
F1 Score : 0.823891226934283


2) Cross-validation (k=5) cho Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=42)

cv_results = cross_validate(
    rf,
    X,
    y,
    cv=5,
    scoring=["accuracy", "precision", "recall", "f1"],
    return_train_score=False
)

pd.DataFrame(cv_results)


3) Hồi quy – MAE, RMSE, R²

Dataset Churn là dạng phân loại, không có mục tiêu dạng số.
Nên ở đây ta chọn biến Tenure để dự đoán MonthlyCharges → tạo bài toán regression.

In [ ]:
reg_df = df_encoded.copy()

target = "MonthlyCharges"
features = reg_df.drop(columns=[target, "Churn"])

X_reg = features
y_reg = reg_df[target]

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

linr = LinearRegression()
linr.fit(X_train_reg, y_train_reg)

y_pred_linr = linr.predict(X_test_reg)


Random Forest Regressor

In [ ]:
rfr = RandomForestRegressor(random_state=42)
rfr.fit(X_train_reg, y_train_reg)

y_pred_rfr = rfr.predict(X_test_reg)


Tính MAE, RMSE, R²

In [ ]:
def regression_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

print("=== Linear Regression ===")
print(regression_metrics(y_test_reg, y_pred_linr))

print("\n=== Random Forest Regressor ===")
print(regression_metrics(y_test_reg, y_pred_rfr))


4) Vì sao mô hình tốt về kỹ thuật chưa chắc tốt về nghiệp vụ?

Mô hình tốt về metric chưa chắc tốt về mặt nghiệp vụ vì:
1. Chi phí sai lầm khác nhau

Ví dụ trong churn prediction:

False Negative (bỏ sót khách sắp rời đi) → mất doanh thu lớn

False Positive (nhầm khách không rời đi) → chỉ tốn tiền giữ chân

Dù accuracy cao, nhưng bỏ sót khách quan trọng vẫn gây thiệt hại.

2. Mục tiêu kinh doanh không phải lúc nào cũng là accuracy

Doanh nghiệp thường quan tâm:

Tối đa hóa lợi nhuận

Giảm chi phí chiến dịch

Tối ưu tệp khách hàng tiềm năng

Metric kỹ thuật không đo được các yếu tố này.

3. Mô hình có thể không giải thích được

Random Forest, XGBoost khó giải thích → không phù hợp khi doanh nghiệp cần:

Giải thích cho quản lý

Lý do để thuyết phục khách hàng

4. Dữ liệu thay đổi theo thời gian

Mô hình tốt hôm nay có thể lỗi thời khi:

Xu hướng khách hàng đổi

Sản phẩm thay đổi

Cạnh tranh mới xuất hiện

5. Metric tốt nhưng mô hình không khả thi để triển khai

Quá nặng → chi phí server cao

Thời gian dự đoán chậm

Không dễ tích hợp với hệ thống hiện tại